In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 第六章 连接
### Ex1：美国疫情数据集

In [6]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('str').astype('string').str.zfill(2) +'-'+ date.dt.day.astype('str').astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()

In [7]:
L = []
for d in date:
    df = pd.read_csv('./data/us_report/' + d + '.csv', index_col='Province_State')
    data = df.loc['New York', ['Confirmed','Deaths','Recovered','Active']]
    L.append(data.to_frame().T)
res = pd.concat(L)
res.index = date
res.head()

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887,179648
04-13-2020,195749,10058,23887,185691
04-14-2020,203020,10842,23887,192178
04-15-2020,214454,11617,23887,202837
04-16-2020,223691,14832,23887,208859


### Ex2：实现join函数

In [8]:
def join(df1, df2, how='left'):
    res_col = df1.columns.tolist() +  df2.columns.tolist()
    dup = df1.index.unique().intersection(df2.index.unique())
    res_df = pd.DataFrame(columns = res_col)
    for label in dup:
        cartesian = [list(i)+list(j) for i in df1.loc[label].values for j in df2.loc[label].values]
        dup_df = pd.DataFrame(cartesian, index = [label]*len(cartesian), columns = res_col)
        res_df = pd.concat([res_df,dup_df])
    if how in ['left', 'outer']:
        for label in df1.index.unique().difference(dup):
            if isinstance(df1.loc[label], pd.DataFrame):
                cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].values]
            else:
                cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].to_frame().values]
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    if how in ['right', 'outer']:
        for label in df2.index.unique().difference(dup):
            if isinstance(df2.loc[label], pd.DataFrame):
                cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].values]
            else:
                cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].to_frame().values]
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    return res_df

In [9]:
df1 = pd.DataFrame({'col1':list('01234')}, index=list('AABCD'))
df1

,col1
A,0
A,1
B,2
C,3
D,4


In [10]:
df2 = pd.DataFrame({'col2':list('opqrst')}, index=list('ABBCEE'))
df2

,col2
A,o
B,p
B,q
C,r
E,s
E,t


In [11]:
join(df1, df2, how='outer')

,col1,col2
A,0,o
A,1,o
B,2,p
B,2,q
C,3,r
D,4,NaN
E,NaN,s
E,NaN,t
